In [2]:
import pandas as pd
import numpy as np
import yahoo_fin.stock_info as yh
from stock_clustering import clusters_dict
from stock_dataset import stock_data
import warnings
from sklearn.preprocessing import StandardScaler

warnings.filterwarnings("ignore")

Stock data ready to use!
Clustering done!


In [3]:
clusters_dict[2] = clusters_dict.pop(4)
clusters_dict.items()

dict_items([(3, ['AAPL', 'HD', 'MCD', 'MSFT', 'PG', 'WMT']), (1, ['AMGN', 'CVX', 'IBM', 'JNJ', 'KO', 'MRK', 'TRV', 'UNH']), (0, ['AXP', 'BA', 'CAT', 'CSCO', 'DOW', 'GS', 'HON', 'JPM', 'V']), (2, ['CRM', 'NKE'])])

Teraz co ja chcę z tym zrobić?
- potrzebuję utworzyć osobne słowniki dfów dla każdego klastra - **ZROBIONE**
- ekperymentalnie, na jednym ze słowników zrobić dwa modele - dla close i dla daily change:
    - najpierw zmienić dane na windowed df dla lstm'a (długość? trzy ostatnie dni starczą chyba); **ZROBIONE**
    - rozdzielić na train val test **ZROBIONE**
    - zbudować model dla close i model dla dailychange
    - porównać modele pod kątem sprawdzania się dla różnych firm
    - jeśli źle, przemyśleć klastryzację
    - jeśli w miarę dobrze, wybrać na czym opieramy model (close czy dailychange)
- stworzyć modele dla wszystkich klastrów
- ocenić ich jakość
- zapisać modele
- sprawdzić, jak je douczać

In [4]:
scaler = StandardScaler()

for key in stock_data:
    stock_data[key].set_index('date', inplace=True)
    stock_data[key][['close', 'dailychange']] = scaler.fit_transform(stock_data[key][['close', 'dailychange']])

In [5]:
stock_data['AAPL']

,close,dailychange
date,,
2020-07-20,-2.330226,-0.055511
2020-07-21,-2.396798,-0.756670
2020-07-22,-2.383435,0.087207
2020-07-23,-2.600561,-2.367859
2020-07-24,-2.611841,-0.181363
...,...,...
2023-07-12,2.152691,0.400977
2023-07-13,2.190452,0.150619
2023-07-14,2.197808,-0.015515


In [6]:
cluster_data = {}
for cluster, firm_names in clusters_dict.items():
    cluster_data[cluster] = {firm_name: stock_data[firm_name] for firm_name in firm_names}

In [7]:
cluster_0 = {}
for firm_name, df in cluster_data[0].items():
    cluster_0[firm_name] = df
    
cluster_1 = {}
for firm_name, df in cluster_data[1].items():
    cluster_1[firm_name] = df
    
cluster_2 = {}
for firm_name, df in cluster_data[2].items():
    cluster_2[firm_name] = df

cluster_3 = {}
for firm_name, df in cluster_data[3].items():
    cluster_3[firm_name] = df

In [8]:
cluster_0['CSCO']

,close,dailychange
date,,
2020-07-20,-0.391614,-0.015302
2020-07-21,-0.382927,0.053929
2020-07-22,-0.403776,-0.181280
2020-07-23,-0.315167,0.691916
2020-07-24,-0.490648,-1.400806
...,...,...
2023-07-12,0.263399,-1.737293
2023-07-13,0.400657,0.997285
2023-07-14,0.200852,-1.466725


In [10]:
closing = {}
for key in cluster_0.keys():
    closing[key] = cluster_0[key]['close']

In [11]:
daily_change = {}
for key in cluster_0.keys():
    daily_change[key] = cluster_0[key]['dailychange']

In [13]:
window = 4

X = {}
Y = {}

for key in closing.keys():
    X[key] = []
    Y[key] = []
    for i in range(window, len(closing[key])):
        X[key].append(closing[key][i-window:i])
        Y[key].append(closing[key][i])

In [14]:
q_80 = int(len(X['CSCO']) * .8)
q_99 = int(len(X['CSCO']) * .99)

X_train, X_val, X_test = {}, {}, {}
Y_train, Y_val, Y_test = {}, {}, {}

for key in X.keys():
    X_train[key], X_val[key], X_test[key] = X[key][:q_80], X[key][q_80:q_99], X[key][q_99:]

for key in Y.keys():
    Y_train[key], Y_val[key], Y_test[key] = Y[key][:q_80], Y[key][q_80:q_99], Y[key][q_99:]                                                


AttributeError: 'list' object has no attribute 'shape'

In [15]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

In [19]:
model=Sequential()
model.add(LSTM(50, activation='relu', input_shape=(4,1)))
model.add(Dense(25))
model.add(Dense(1))

In [20]:
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 50)                10400     
                                                                 
 dense_2 (Dense)             (None, 25)                1275      
                                                                 
 dense_3 (Dense)             (None, 1)                 26        
                                                                 
Total params: 11701 (45.71 KB)
Trainable params: 11701 (45.71 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
for key in X.keys():
    model.fit(X_train[key], Y_train[key], epochs=10, batch_size=10)

ValueError: Failed to find data adapter that can handle input: (<class 'list'> containing values of types {"<class 'pandas.core.series.Series'>"}), (<class 'list'> containing values of types {"<class 'numpy.float64'>"})